In [6]:
import numpy as np
from operation import Operation

xmatrix = np.array([[1, 2, 3], [np.nan, 5, np.nan], [7, 8, 9], [10, 11, 12]])
ymatrix = np.array([[np.nan, np.nan, np.nan], [4, np.nan, 6], [7, 8, 9], [10, 11, 12]])
window_size = 2
res1 = Operation().rolling_argmax(xmatrix, 2)
res2 = Operation().rolling_argmin(xmatrix, 2)

In [7]:
print(res1)
print(res2)

[[0. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 1. 1.]
 [0. 1. 0.]
 [1. 1. 1.]]
